In [ ]:
import torch
import os

print("CUDA available:", torch.cuda.is_available())
print("Device count:", torch.cuda.device_count())
print("Device name(s):", [torch.cuda.get_device_name(i) for i in range(torch.cuda.device_count())])
# Get Slurm-allocated CPU count or fallback
n_jobs = int(os.environ.get("SLURM_CPUS_PER_TASK", os.cpu_count()))
print("Number of jobs:", n_jobs)
n_human = int(n_jobs/4)

In [ ]:
params = {
    'model': 'lorenz',
    'case': 'synthetic_lorenz',
    'input_dim': 80,
    'latent_dim': 3,
    'poly_order': 2,
    'include_sine': False,
    'fix_coefs': False,
    'svd_dim': 40,
    'delay_embedded': True,
    'scale': True,
    'coefficient_initialization': 'constant',
    "coefficient_initial_value": 1e-2,
    'widths_ratios': [1.0, 0.6],

    # Training
    'max_epochs': 500,
    'patience': 20,
    'batch_size': 2**9,
    'learning_rate': 1e-3,
    "lr_decay": 0.999,

    # Loss Weights
    'loss_weight_rec': 0.3,
    'loss_weight_sindy_z': 0.001,
    'loss_weight_sindy_x': 0.001,
    'loss_weight_sindy_regularization': 1e-5,
    'loss_weight_integral': 0.1,
    'loss_weight_x0': 0.01,
    'loss_weight_layer_l2': 0.0,
    'loss_weight_layer_l1': 0.0,

    # SINDy
    'coefficient_threshold': 1e-5,
    'threshold_frequency': 5,
    'print_frequency': 10,
    'sindy_pert': 0.0,
    'ode_net': False,
    'ode_net_widths': [1.5, 2.0],
    'exact_features': True,
    'use_bias': True,

    # Misc
    'train_ratio': 0.8,
    'data_path': './trained_models',
    'save_checkpoints': False,
    'save_freq': 5,
    'learning_rate_sched': False,
    'use_sindycall': False,
    'sparse_weighting': None,
    'system_coefficients': None
}

In [ ]:
import torch
import time
from TrainingCode.TrainingCode import TrainModel  # adjust import path if needed
from ParamOptCode.DefaultParams import params
from AdditionalFunctionsCode.SolverFunctions import SynthData
from DataGenerationCode.DataGeneration import LorenzSystem

# === SETUP ===
params["max_epochs"] = 500
params["input_dim"] = params["svd_dim"]  # ensure consistent dimensions
params["batch_size"] = 2**9  # larger batch to leverage GPU better
params["dt"] = 0.01 

# === Generate synthetic data ===
lorenz_system = LorenzSystem()
synth_data = SynthData(input_dim=params["input_dim"])
z0 = torch.tensor([1.0, 1.0, 1.0])
synth_data.run_sim(tend=32, dt=0.01, z0=z0.numpy(), apply_svd=True, svd_dim=params["svd_dim"])

# === Choose device ===
# Change to 'cpu' to benchmark that version
#device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = torch.device("cpu")  # Uncomment to test CPU explicitly

# === Train ===
trainer = TrainModel(synth_data, params, device=device)

start = time.time()
trainer.fit()
end = time.time()

device_type = "GPU" if device.type == "cuda" else "CPU"
print(f"\n=== Training on {device_type} ({device}) completed in {(end - start)/60:.2f} minutes ===")

In [ ]:
print(params["svd_dim"])

In [1]:
# Delete old database
import os
import torch
os.remove("optuna_study.db")  # or your full path

Perform the Hyperparameter Optimization

In [2]:
from ParamOptCode.ParameterOptimizer import HyperparameterOptimizer
from ParamOptCode.DefaultParams import params 
import time
#os.remove('C:/Users/mehme/OneDrive/Desktop/All/Code/LorenzReclaim/optuna_study.db')
start_time = time.time()
device = torch.device("cpu")
device_type = "GPU" if device.type == "cuda" else "CPU"
params["input_dim"] = params["svd_dim"]
optimizer = HyperparameterOptimizer(n_trials=2, n_jobs=1)
best = optimizer.run()
end_time = time.time()
print(f"Best hyperparameters: {best}")
print(f"Hyperparameter optimization completed in {(end_time - start_time)/60:.2f} minutes")

[I 2025-05-11 23:56:47,325] A new study created in RDB with name: sindy_opt


Generating Lorenz system...
[DEBUG] Final coefficient_threshold: 2.201089143479006e-06
[DEBUG] Final rfe_threshold: 2.201089143479006e-06
[DEBUG] Sparse weighting initialized with shape: (5, 3)

[DEBUG] SINDy fixed_mask in params:
[[0 0 0]
 [0 0 0]
 [0 0 0]
 [0 0 0]
 [0 0 0]
 [0 0 0]
 [0 0 0]
 [0 0 0]
 [0 0 0]
 [0 0 0]]
[DEBUG] SINDy fixed_values in params:
[[ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0. -1.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0. -1.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]]
[DEBUG] rfe_threshold at model creation: 2.201089143479006e-06
[DEBUG] Sindy rfe_threshold set to: 2.201089143479006e-06

Active Loss Weights:
loss_weight_rec: 0.3346683838008026
loss_weight_sindy_z: 5.3908677752127535e-06
loss_weight_sindy_x: 0.007882446114526823
loss_weight_sindy_regularization: 0.04966276141994712
loss_weight_integral: 0.0037255017611278088
loss_weight_x0: 0.010208036891739165
loss_weight_layer_l2: 0.009684699938938305
loss_weight_layer_l1: 0.00562718823435104
Laun

[I 2025-05-12 00:06:46,725] Trial 0 finished with value: 674.2346801757812 and parameters: {'sindy_mask_type': 1, 'loss_weight_rec': 0.3346683838008026, 'loss_weight_sindy_z': 5.3908677752127535e-06, 'loss_weight_sindy_x': 0.007882446114526823, 'loss_weight_sindy_regularization': 0.04966276141994712, 'loss_weight_integral': 0.0037255017611278088, 'loss_weight_x0': 0.010208036891739165, 'loss_weight_layer_l2': 0.009684699938938305, 'loss_weight_layer_l1': 0.00562718823435104, 'learning_rate': 0.0008091408620185395, 'lr_decay': 0.9950047754040761, 'batch_size': 128, 'coefficient_threshold': 2.201089143479006e-06}. Best is trial 0 with value: 674.2346801757812.



Final SINDy Model:
dz0/dt = 0.0371 * 1 + 0.0951 * z0 + -0.2972 * z1 + 0.2509 * z2 + 0.0077 * z0z0 + -0.0068 * z0z1 + 0.0304 * z0z2 + 0.0466 * z1z1 + 0.0099 * z1z2 + 0.0127 * z2z2
dz1/dt = -0.0225 * 1 + -0.0197 * z0 + 0.0960 * z1 + -0.1068 * z2 + -0.0468 * z0z0 + 0.1000 * z0z1 + -0.0856 * z0z2 + -0.0620 * z1z1 + 0.0218 * z1z2 + -0.0272 * z2z2
dz2/dt = -0.0425 * 1 + -0.0064 * z0 + -0.1706 * z1 + 0.2541 * z2 + 0.0086 * z0z0 + -0.0822 * z0z1 + 0.0482 * z0z2 + 0.0418 * z1z1 + -0.0164 * z1z2 + -0.0090 * z2z2
Generating Lorenz system...
[DEBUG] Final coefficient_threshold: 7.431526208458673e-06
[DEBUG] Final rfe_threshold: 7.431526208458673e-06
[DEBUG] Sparse weighting initialized with shape: (5, 3)

[DEBUG] SINDy fixed_mask in params:
[[0 0 0]
 [0 0 0]
 [0 1 0]
 [0 0 0]
 [0 0 0]
 [0 0 0]
 [0 1 0]
 [1 1 1]
 [1 1 1]
 [1 1 1]]
[DEBUG] SINDy fixed_values in params:
[[ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0. -1.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0. -1.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  

[I 2025-05-12 00:16:49,192] Trial 1 finished with value: 277.48919677734375 and parameters: {'sindy_mask_type': 0, 'loss_weight_rec': 0.0027612799850701276, 'loss_weight_sindy_z': 0.03586389936865883, 'loss_weight_sindy_x': 0.03816846195737673, 'loss_weight_sindy_regularization': 2.751430332144227e-06, 'loss_weight_integral': 0.6806079044959886, 'loss_weight_x0': 0.3870283764443174, 'loss_weight_layer_l2': 0.009531817684726164, 'loss_weight_layer_l1': 0.0013425648788334189, 'learning_rate': 1.4895495026798487e-05, 'lr_decay': 0.9986771473827022, 'batch_size': 128, 'coefficient_threshold': 7.431526208458673e-06}. Best is trial 1 with value: 277.48919677734375.



Final SINDy Model:
dz0/dt = -0.1034 * 1 + 0.0868 * z0 + -0.1215 * z1 + 0.1285 * z2 + -0.1232 * z0z0 + 0.0971 * z0z1 + -0.1048 * z0z2
dz1/dt = 0.1248 * 1 + -0.0801 * z0 + -1.0000 * z1 + -0.1436 * z2 + 0.1445 * z0z0 + -0.0878 * z0z1 + -1.0000 * z0z2
dz2/dt = 0.0100 * 1 + 0.0745 * z0 + -0.0193 * z1 + -0.0248 * z2 + -0.0608 * z0z0 + 0.0826 * z0z1 + -0.0934 * z0z2

Top 5 Trials by Static Loss:

📌 Trial 1 (Global Trial #1):
  Static Loss: 277.489197
  Params:
    sindy_mask_type: 0
    loss_weight_rec: 0.0027612799850701276
    loss_weight_sindy_z: 0.03586389936865883
    loss_weight_sindy_x: 0.03816846195737673
    loss_weight_sindy_regularization: 2.751430332144227e-06
    loss_weight_integral: 0.6806079044959886
    loss_weight_x0: 0.3870283764443174
    loss_weight_layer_l2: 0.009531817684726164
    loss_weight_layer_l1: 0.0013425648788334189
    learning_rate: 1.4895495026798487e-05
    lr_decay: 0.9986771473827022
    batch_size: 128
    coefficient_threshold: 7.431526208458673e-06

 